In [ ]:
import numpy as np
import tensorflow.keras as keras
import tensorflow as tf

LATENT_SIZE = 128

inputs = keras.Input((28, 28, 1))
x = keras.layers.Conv2D(64, (3, 3), strides=(2, 2), activation="selu", padding="same")(inputs)
x = keras.layers.MaxPooling2D((2, 2), padding="same")(x)
x = keras.layers.Conv2D(64, (3, 3), activation="selu", padding="same")(x)
x = keras.layers.MaxPooling2D(2, 2, padding="same")(x)
x = keras.layers.Flatten()(x)
x = keras.layers.Dense(4 * 4 * 16, activation="selu")(x)
latent = keras.layers.Dense(LATENT_SIZE, activation="sigmoid")(x)

latent_inputs = keras.Input(LATENT_SIZE)
x = keras.layers.Dense(4 * 4 * 16, activation="selu")(latent_inputs)
x = keras.layers.Reshape((4, 4, 16))(x)
x = keras.layers.Conv2DTranspose(64, (4, 4), strides=1, activation="selu", padding="valid")(x)
x = keras.layers.Conv2DTranspose(64, (3, 3), strides=2, activation="selu", padding="same")(x)
x = keras.layers.Conv2DTranspose(64, (2, 2), strides=2, activation="selu", padding="same")(x)
x = keras.layers.Conv2DTranspose(64, (3, 3), strides=1, activation="selu", padding="same")(x)
ae_outputs = keras.layers.Conv2D(1, (3, 3), activation="sigmoid", padding="same")(x)

encoder = keras.Model(inputs, latent)

decoder = keras.Model(latent_inputs, ae_outputs)

enc_dec = encoder(inputs)

ae_model_outputs = decoder(enc_dec)

ae_model = keras.Model(inputs, ae_model_outputs)

ae_model.compile(loss=keras.losses.BinaryCrossentropy(), optimizer=keras.optimizers.Adam())


In [ ]:
import tensorflow_datasets as tfds

(x_train, y_train), (x_test, y_test) = keras.datasets.fashion_mnist.load_data()

x_train = x_train.astype("float32") / 255
x_test = x_test.astype("float32") / 255
# Make sure images have shape (28, 28, 1)
x_train = np.expand_dims(x_train, -1)
x_test = np.expand_dims(x_test, -1)

In [ ]:
ae_model.fit(x_train, x_train, epochs=6, validation_data=(x_test, x_test), batch_size=256)

In [ ]:
encoder.trainable = False
decoder.trainable = False

In [ ]:
res = ae_model(x_test[:4])

In [ ]:
import matplotlib.pyplot as plt
plt.axis("off")
plt.imshow(x_test[2], cmap="gray")

In [ ]:
plt.axis("off")
plt.imshow(res[2], cmap="gray")

In [ ]:
ae_model.trainable = False

inputs = keras.Input((28, 28, 1))
x = ae_model(inputs)
x = keras.layers.Conv2D(16, (3, 3), strides=(2, 2), activation="selu", padding="same")(x)
x = keras.layers.MaxPooling2D((2, 2), padding="same")(x)
x = keras.layers.Conv2D(16, (3, 3), activation="selu", padding="same")(x)
x = keras.layers.MaxPooling2D((2, 2), padding="same")(x)
x = keras.layers.Conv2D(16, (3, 3), activation="selu", padding="same")(x)
intermediate = keras.layers.Flatten()(x)
x = keras.layers.MaxPooling2D((2, 2), padding="same")(x)
x = keras.layers.Flatten()(x)
outputs = keras.layers.Dense(10, activation="selu")(x)
outputs = keras.layers.Softmax()(outputs)
classifier = keras.Model(inputs, outputs)
classifier.compile(loss=keras.losses.SparseCategoricalCrossentropy(), optimizer=keras.optimizers.Adam(), metrics=["accuracy"])

classifier_intermediate = keras.Model(inputs, intermediate)

In [ ]:
classifier.fit(x_train, y_train, epochs=6, validation_data=(x_test, y_test), batch_size=256)

In [ ]:
concept_index = []
for case in x_test:
    case = case.flatten()
    concept_index.append((case > 0.7).sum() / (case > 0).sum())
concept_index = np.array(concept_index)

In [ ]:
intermediate_outputs = classifier_intermediate(inputs)

probe_output = keras.layers.Dense(1, activity_regularizer=keras.regularizers.L1(0.1), activation="sigmoid")(intermediate_outputs)

probe = keras.Model(inputs, probe_output)
probe.compile(loss=keras.losses.MeanSquaredError(), optimizer=keras.optimizers.Adam())

probe_x_train, probe_y_train, probe_x_test, probe_y_test = x_test[:7000], concept_index[:7000], x_test[7000:], concept_index[7000:]

In [ ]:
probe.fit(probe_x_train, probe_y_train, validation_data=(probe_x_test, probe_y_test), epochs=17)

In [ ]:
encoded_image = encoder(inputs)
maximiser_input = keras.Input(1)
max_plane = keras.layers.Dense(LATENT_SIZE, name="max_plane", activation="linear")(maximiser_input)
additive = keras.layers.Add()([max_plane, encoded_image])
image_output = decoder(additive)
image_output._name = "image_output"
intermediate_output = classifier_intermediate(image_output)
maximised_probe_output = keras.layers.Dense(1, name="max_probe_repl", activity_regularizer=keras.regularizers.L1(0.01), activation="sigmoid")(intermediate_output)
maximiser_model = keras.Model([inputs, maximiser_input], [maximised_probe_output, image_output])
decoder.trainable = False
encoder.trainable = False
classifier_intermediate.trainable = False
look_at_results_output = decoder(additive)
look_at_results_model = keras.Model([inputs, maximiser_input], look_at_results_output)

maximiser_model.get_layer("max_probe_repl").set_weights(probe.layers[-1].get_weights())
maximiser_model.get_layer("max_probe_repl").trainable = False

maximiser_model.get_layer("max_plane").trainable = True

maximiser_model.compile(loss=["binary_crossentropy", "mean_squared_error"], loss_weights=[1, 5], optimizer=keras.optimizers.Adam())

In [ ]:
# 95, 41, 150, 777

case = np.repeat(np.expand_dims(probe_x_test[41], axis=0), 64, axis=0)
ones = np.expand_dims(np.ones(64), -1)

In [ ]:
import matplotlib as mpl
import matplotlib.style

mpl.rcParams['figure.dpi'] = 100
plt.imshow(case[0])

In [ ]:
maximiser_model.fit([case, ones], [ones, case], epochs=600, batch_size=32)

In [ ]:
import matplotlib as mpl
import matplotlib.style

mpl.style.use("seaborn-muted")
mpl.rcParams['figure.dpi'] = 300
mpl.rcParams['font.family'] = "serif"

_, image = maximiser_model([case, ones])

_, axs = plt.subplots(1, 2, figsize=(12, 12))
axs[1].axis("off")
axs[1].imshow(np.clip(image[0], 0.1, 1), cmap="gray")
axs[1].set_title("Maximised", fontdict={"fontsize": 35})

axs[0].axis("off")
axs[0].imshow(case[0], cmap="gray")
axs[0].set_title("Normal image", fontdict={"fontsize": 35})
